In [1]:
import pandas as pd 

df = pd.read_csv("symtab.csv")
df.head()

,Scope,Type,Variable Name,Memloc,Line Numbers
0,global,void,main,24,34
1,global,void,sort,17,21 42
2,global,int,vet,0,4 39 42 45
3,global,int,minloc,10,6 26
4,minloc,int,low,12,6 8 9 10


In [2]:
def getMemLoc(row):
    if "global" in row["Scope"]:
        return row["Memloc"]
    else:
        return row["Memloc"] - df.loc[df["Variable Name"] == row["Scope"],"Memloc"].values[0] - 1

df["Scope Memloc"] = df.apply(getMemLoc, axis=1)
df.sort_values(by='Memloc')


,Scope,Type,Variable Name,Memloc,Line Numbers,Scope Memloc
2,global,int,vet,0,4 39 42 45,0
3,global,int,minloc,10,6 26,10
5,minloc,int,a,11,6 9 12 13,0
4,minloc,int,low,12,6 8 9 10,1
9,minloc,int,high,13,6 11,2
6,minloc,int,i,14,7 10 11 12 13 14 16 16,3
8,minloc,int,x,15,7 9 12 13,4
7,minloc,int,k,16,7 8 14 18,5
1,global,void,sort,17,21 42,17
11,sort,int,a,18,21 26 27 28 28 29,0


In [9]:
reg = [None]*25

with open("sort.asm", "w") as asm:

    with open("../sort.ic", "r") as f:
        scope = 'global'
        line = f.readline().strip()
        while line != '':
            quad = line.split(" ")

            for i in quad:
                if "$t" in i: 
                    aux = i.split("(")[1][:-1] if "(" in i else i
                    if not aux in reg: reg[reg.index(None)]=aux

            if  quad[0] == "FUN":
                scope = quad[2]
                asm.write(f".{quad[2]}\n")

            elif quad[0] == "ALLOC":
                asm.write("ADDI $r30 $r30 1\n")

            elif quad[0] == "ARG":
                asm.write("ADDI $r30 $r30 1\n")

            elif quad[0] == "LOAD":
                scopevar = scope

                if "(" in quad[2]: 
                    x = 3
                    var = quad[2].split('(')[0]
                else: 
                    x = 0
                    var = quad[2]

                if not df.loc[df['Scope'] == scopevar, 'Variable Name'].isin([var]).any(): scopevar = 'global' 
                asm.write(f"LW $t0 {df.loc[((df['Scope'] == scopevar) & (df['Variable Name'] == var)), 'Scope Memloc'].values[0] + x}\n")
            
            elif quad[0] == "LAB":
                asm.write(f".{quad[1]}\n")

            elif quad[0] == "MAIS":
                asm.write(f"ADD $t{reg.index(quad[1])} $t{reg.index(quad[2])} $t{reg.index(quad[3])}\n")

            elif quad[0] == "MENOS":
                asm.write(f"SUB $t{reg.index(quad[1])} $t{reg.index(quad[2])} $t{reg.index(quad[3])}\n")

            elif quad[0] == "VEZES":
                asm.write(f"MULT $t{reg.index(quad[1])} $t{reg.index(quad[2])} $t{reg.index(quad[3])}\n")
            
            elif quad[0] == "SOBRE":
                asm.write(f"DIV $t{reg.index(quad[1])} $t{reg.index(quad[2])} $t{reg.index(quad[3])}\n")

            elif quad[0] == "MENORQ":
                asm.write(f"LT $t{reg.index(quad[1])} $t{reg.index(quad[2])} $t{reg.index(quad[3])}\n")

            elif quad[0] == "MENORI":
                asm.write(f"LE $t{reg.index(quad[1])} $t{reg.index(quad[2])} $t{reg.index(quad[3])}\n")

            elif quad[0] == "MAIORQ":
                asm.write(f"GT $t{reg.index(quad[1])} $t{reg.index(quad[2])} $t{reg.index(quad[3])}\n")

            elif quad[0] == "MAIORI":
                asm.write(f"GE $t{reg.index(quad[1])} $t{reg.index(quad[2])} $t{reg.index(quad[3])}\n")

            elif quad[0] == "IGUAL":
                asm.write(f"EQ $t{reg.index(quad[1])} $t{reg.index(quad[2])} $t{reg.index(quad[3])}\n")
            
            elif quad[0] == "DIF":
                asm.write(f"DF $t{reg.index(quad[1])} $t{reg.index(quad[2])} $t{reg.index(quad[3])}\n")
            
            elif quad[0] == "IFF":
                asm.write(f"IFF $t{reg.index(quad[1])} {quad[2]}\n")

            elif quad[0] == "GOTO":
                asm.write(f"J {quad[1]}\n")

            elif quad[0] == "END":
                asm.write(f".FIMFUN {quad[1]}\n")

            elif quad[0] == "HALT":
                asm.write(f"HALT\n")

            else: asm.write(f"Unknown - {quad[0]}\n")

            pos = f.tell()
            index = []
            nextLine = f.readline().strip()
            while nextLine != "":
                for i, r in enumerate(reg):
                    if r is not None and r in nextLine and i not in index: index.append(i)
                nextLine = f.readline().strip()

            for i in range(len(reg)):
                if i not in index: reg[i] = None

            f.seek(pos)
            line = f.readline().strip()
